In [285]:
''''In tis main file we do an initial explortation, we clean the data, and perfoirm EDA and vizualisation'''

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import initial_exploration as explo
import data_cleaning as cl

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [286]:
# Load datasets:
cash_requests = pd.read_csv('project_dataset/extract - cash request - data analyst.csv')
fees = pd.read_csv('project_dataset/extract - fees - data analyst - .csv')

In [287]:
cash_requests.head()

,id,amount,status,created_at,updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update
0,5,100.0,rejected,2019-12-10 19:05:21.596873+00,2019-12-11 16:47:42.40783+00,804.0,2019-12-11 16:47:42.405646+00,NaN,2020-01-09 19:05:21.596363+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
1,70,100.0,rejected,2019-12-10 19:50:12.34778+00,2019-12-11 14:24:22.900054+00,231.0,2019-12-11 14:24:22.897988+00,NaN,2020-01-09 19:50:12.34778+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
2,7,100.0,rejected,2019-12-10 19:13:35.82546+00,2019-12-11 09:46:59.779773+00,191.0,2019-12-11 09:46:59.777728+00,NaN,2020-01-09 19:13:35.825041+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
3,10,99.0,rejected,2019-12-10 19:16:10.880172+00,2019-12-18 14:26:18.136163+00,761.0,2019-12-18 14:26:18.128407+00,NaN,2020-01-09 19:16:10.879606+00,NaN,NaN,regular,NaN,NaN,NaN,NaN
4,1594,100.0,rejected,2020-05-06 09:59:38.877376+00,2020-05-07 09:21:55.34008+00,7686.0,2020-05-07 09:21:55.320193+00,NaN,2020-06-05 22:00:00+00,NaN,NaN,regular,NaN,NaN,NaN,NaN


In [288]:
# Inital exploration:
explo.check(cash_requests)

Number of columns: 16 amd rows: 23970

Data types:
id                              int64
amount                        float64
status                         object
created_at                     object
updated_at                     object
user_id                       float64
moderated_at                   object
deleted_account_id            float64
reimbursement_date             object
cash_request_received_date     object
money_back_date                object
transfer_type                  object
send_at                        object
recovery_status                object
reco_creation                  object
reco_last_update               object
dtype: object

Unique values count:
id                            23970
amount                           41
status                            7
created_at                    23970
updated_at                    23970
user_id                       10798
moderated_at                  16035
deleted_account_id             1141
reimbursement_dat

In [289]:
cl.convert_dates(cash_requests, cl.cash_request_date_columns)

# Re-check and ensure correct data types in cash_requests:
cl.ensure_correct_data_types(cash_requests, cl.cash_request_date_columns)

cash_requests.head()

,id,amount,status,created_at,updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update
0,5,100.0,rejected,2019-12-10,2019-12-11,804.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT
1,70,100.0,rejected,2019-12-10,2019-12-11,231.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT
2,7,100.0,rejected,2019-12-10,2019-12-11,191.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT
3,10,99.0,rejected,2019-12-10,2019-12-18,761.0,2019-12-18,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT
4,1594,100.0,rejected,2020-05-06,2020-05-07,7686.0,2020-05-07,NaN,2020-06-05,NaT,NaT,regular,NaT,NaN,NaT,NaT


In [290]:
explo.check(cash_requests)

Number of columns: 16 amd rows: 23970

Data types:
id                                     int64
amount                               float64
status                              category
created_at                    datetime64[ns]
updated_at                    datetime64[ns]
user_id                              float64
moderated_at                  datetime64[ns]
deleted_account_id                   float64
reimbursement_date            datetime64[ns]
cash_request_received_date    datetime64[ns]
money_back_date               datetime64[ns]
transfer_type                       category
send_at                       datetime64[ns]
recovery_status                     category
reco_creation                 datetime64[ns]
reco_last_update              datetime64[ns]
dtype: object

Unique values count:
id                            23970
amount                           41
status                            7
created_at                      330
updated_at                      401
user_id      

In [291]:
fees.head()

,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
0,6537,14941.0,instant_payment,rejected,NaN,5.0,Instant Payment Cash Request 14941,2020-09-07 10:47:27.42315+00,2020-10-13 14:25:09.396112+00,2020-12-17 14:50:07.47011+00,NaN,NaN,after
1,6961,11714.0,incident,accepted,rejected_direct_debit,5.0,rejected direct debit,2020-09-09 20:51:17.998653+00,2020-10-13 14:25:15.537063+00,2020-12-08 17:13:10.45908+00,NaN,NaN,after
2,16296,23371.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 23371,2020-10-23 10:10:58.352972+00,2020-10-23 10:10:58.352994+00,2020-11-04 19:34:37.43291+00,NaN,NaN,after
3,20775,26772.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 26772,2020-10-31 15:46:53.643958+00,2020-10-31 15:46:53.643982+00,2020-11-19 05:09:22.500223+00,NaN,NaN,after
4,11242,19350.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 19350,2020-10-06 08:20:17.170432+00,2020-10-13 14:25:03.267983+00,2020-11-02 14:45:20.355598+00,NaN,NaN,after


In [292]:
explo.check(fees)

Number of columns: 13 amd rows: 21061

Data types:
id                   int64
cash_request_id    float64
type                object
status              object
category            object
total_amount       float64
reason              object
created_at          object
updated_at          object
paid_at             object
from_date           object
to_date             object
charge_moment       object
dtype: object

Unique values count:
id                 21061
cash_request_id    12933
type                   3
status                 4
category               2
total_amount           2
reason             15149
created_at         21026
updated_at         21061
paid_at            15529
from_date           1084
to_date             1560
charge_moment          2
dtype: int64

This columns apear to be categroical:
 Index(['type', 'status', 'category', 'total_amount', 'charge_moment'], dtype='object')

Unique value count for categorical columns:
type
instant_payment    11099
postpone            77

In [293]:
cl.convert_dates(fees, cl.fees_data_date_columns)

# Re-check and ensure correct data types in cfees:
cl.ensure_correct_data_types(fees, cl.fees_data_date_columns)

fees.head()

,id,cash_request_id,type,status,category,total_amount,reason,created_at,updated_at,paid_at,from_date,to_date,charge_moment
0,6537,14941.0,instant_payment,rejected,NaN,5.0,Instant Payment Cash Request 14941,2020-09-07,2020-10-13,2020-12-17,NaT,NaT,after
1,6961,11714.0,incident,accepted,rejected_direct_debit,5.0,rejected direct debit,2020-09-09,2020-10-13,2020-12-08,NaT,NaT,after
2,16296,23371.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 23371,2020-10-23,2020-10-23,2020-11-04,NaT,NaT,after
3,20775,26772.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 26772,2020-10-31,2020-10-31,2020-11-19,NaT,NaT,after
4,11242,19350.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 19350,2020-10-06,2020-10-13,2020-11-02,NaT,NaT,after


In [294]:
cl.rename_col(cash_requests, 'id', 'cash_request_id')
data_df = cl.merge_df(cash_requests, fees, 'outer', 'cash_request_id')
data_df = cl.rename_col_xy(data_df)
data_df.head(20)

,cash_request_id,amount,CR_status,CR_created_at,CR_updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update,id,type,fee_status,category,total_amount,reason,fee_created_at,fee_updated_at,paid_at,from_date,to_date,charge_moment
0,5.0,100.0,rejected,2019-12-10,2019-12-11,804.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
1,70.0,100.0,rejected,2019-12-10,2019-12-11,231.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
2,7.0,100.0,rejected,2019-12-10,2019-12-11,191.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
3,10.0,99.0,rejected,2019-12-10,2019-12-18,761.0,2019-12-18,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
4,1594.0,100.0,rejected,2020-05-06,2020-05-07,7686.0,2020-05-07,NaN,2020-06-05,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
5,2145.0,100.0,money_back,2020-05-23,2020-07-06,9489.0,2020-05-24,NaN,2020-06-06,2020-05-26,2020-07-06,regular,NaT,completed,2020-06-12,2020-07-06,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
6,3512.0,100.0,rejected,2020-06-16,2020-06-17,14631.0,2020-06-17,NaN,2020-07-11,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
7,654.0,100.0,rejected,2020-02-10,2020-02-10,NaN,2020-02-10,309.0,2020-03-11,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
8,5672.0,100.0,canceled,2020-06-28,2020-06-28,NaN,NaT,2499.0,2020-07-07,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
9,71.0,90.0,rejected,2019-12-10,2019-12-12,897.0,2019-12-12,NaN,2019-12-17,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN


In [295]:
data_df.sample(10)

,cash_request_id,amount,CR_status,CR_created_at,CR_updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update,id,type,fee_status,category,total_amount,reason,fee_created_at,fee_updated_at,paid_at,from_date,to_date,charge_moment
22802,17777.0,60.0,money_back,2020-09-25,2020-12-18,49947.0,2020-09-25,NaN,2020-10-06,2020-09-28,2020-10-11,instant,2020-10-02,NaN,NaT,NaT,9263.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 17777,2020-09-25,2020-10-13,2020-10-06,NaT,NaT,after
17621,8902.0,100.0,money_back,2020-07-19,2020-12-18,8496.0,2020-07-20,NaN,2020-08-05,2020-07-24,2020-08-05,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
3210,3705.0,100.0,direct_debit_rejected,2020-06-17,2020-12-08,16730.0,2020-06-18,NaN,2020-07-05,2020-06-20,2020-07-14,regular,NaT,pending,2020-07-15,2020-12-08,1043.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 3705,2020-07-04,2020-10-13,NaT,2020-07-05,2020-08-05,after
1956,4686.0,100.0,rejected,2020-06-22,2020-06-30,19655.0,2020-06-23,NaN,2020-07-06,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
28933,26724.0,25.0,money_back,2020-10-31,2020-12-18,95562.0,NaT,NaN,2020-11-14,2020-11-03,2020-12-04,instant,2020-10-31,completed,2020-11-25,2020-12-04,20704.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 26724,2020-10-31,2020-10-31,2020-12-04,NaT,NaT,after
26161,21673.0,100.0,money_back,2020-10-16,2020-12-18,70091.0,NaT,NaN,2020-11-03,2020-10-20,2020-11-03,instant,2020-10-16,NaN,NaT,NaT,14117.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 21673,2020-10-16,2020-10-16,2020-11-03,NaT,NaT,after
31663,18879.0,100.0,money_back,2020-10-02,2020-12-18,24091.0,2020-10-02,NaN,2020-12-05,2020-10-03,2020-12-04,instant,2020-10-09,NaN,NaT,NaT,11903.0,postpone,accepted,NaN,5.0,Postpone Cash Request 18879,2020-10-08,2020-10-13,2020-10-08,2020-11-05,2020-12-05,before
13730,6341.0,100.0,money_back,2020-07-02,2020-12-18,18047.0,2020-07-03,NaN,2020-08-01,2020-07-07,2020-08-02,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
21781,17019.0,30.0,money_back,2020-09-20,2020-12-18,39840.0,2020-09-21,NaN,2020-10-08,2020-09-22,2020-10-13,instant,2020-09-27,NaN,NaT,NaT,12652.0,postpone,cancelled,NaN,5.0,Postpone Cash Request 17019,2020-10-11,2020-10-13,NaT,2020-10-08,2020-11-07,after
29996,21734.0,50.0,money_back,2020-10-17,2020-12-18,71775.0,NaT,NaN,2020-11-06,2020-10-19,2020-11-06,instant,2020-10-18,NaN,NaT,NaT,14604.0,instant_payment,accepted,NaN,5.0,Instant Payment Cash Request 21734,2020-10-18,2020-10-18,2020-11-06,NaT,NaT,after


In [296]:
cl.rename_col(data_df, 'id', 'id_fee')
cl.clean_text_column(data_df, "reason")
explo.check(data_df)

Number of columns: 28 amd rows: 32098

Data types:
cash_request_id                      float64
amount                               float64
CR_status                           category
CR_created_at                 datetime64[ns]
CR_updated_at                 datetime64[ns]
user_id                              float64
moderated_at                  datetime64[ns]
deleted_account_id                   float64
reimbursement_date            datetime64[ns]
cash_request_received_date    datetime64[ns]
money_back_date               datetime64[ns]
transfer_type                       category
send_at                       datetime64[ns]
recovery_status                     category
reco_creation                 datetime64[ns]
reco_last_update              datetime64[ns]
id_fee                               float64
type                                category
fee_status                          category
category                            category
total_amount                         float64
reas

In [297]:
data_df = cl.remove_nan(data_df, "cash_request_id")
explo.check_null(data_df)

4 rows were removed

Count of null values:
cash_request_id                   0
amount                            0
CR_status                         0
CR_created_at                     0
CR_updated_at                     0
user_id                        2572
moderated_at                  10335
deleted_account_id            29521
reimbursement_date                0
cash_request_received_date     7945
money_back_date                8177
transfer_type                     0
send_at                        9724
recovery_status               24894
reco_creation                 24894
reco_last_update              24894
id_fee                        11037
type                          11037
fee_status                    11037
category                      29898
total_amount                  11037
reason                        11037
fee_created_at                11037
fee_updated_at                11037
paid_at                       16656
from_date                     24328
to_date              

In [298]:
data_df = cl.drop_col(data_df, ["id_fee", "category"])
data_df.head(10)

,cash_request_id,amount,CR_status,CR_created_at,CR_updated_at,user_id,moderated_at,deleted_account_id,reimbursement_date,cash_request_received_date,money_back_date,transfer_type,send_at,recovery_status,reco_creation,reco_last_update,type,fee_status,total_amount,reason,fee_created_at,fee_updated_at,paid_at,from_date,to_date,charge_moment
0,5.0,100.0,rejected,2019-12-10,2019-12-11,804.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
1,70.0,100.0,rejected,2019-12-10,2019-12-11,231.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
2,7.0,100.0,rejected,2019-12-10,2019-12-11,191.0,2019-12-11,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
3,10.0,99.0,rejected,2019-12-10,2019-12-18,761.0,2019-12-18,NaN,2020-01-09,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
4,1594.0,100.0,rejected,2020-05-06,2020-05-07,7686.0,2020-05-07,NaN,2020-06-05,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
5,2145.0,100.0,money_back,2020-05-23,2020-07-06,9489.0,2020-05-24,NaN,2020-06-06,2020-05-26,2020-07-06,regular,NaT,completed,2020-06-12,2020-07-06,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
6,3512.0,100.0,rejected,2020-06-16,2020-06-17,14631.0,2020-06-17,NaN,2020-07-11,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
7,654.0,100.0,rejected,2020-02-10,2020-02-10,NaN,2020-02-10,309.0,2020-03-11,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
8,5672.0,100.0,canceled,2020-06-28,2020-06-28,NaN,NaT,2499.0,2020-07-07,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN
9,71.0,90.0,rejected,2019-12-10,2019-12-12,897.0,2019-12-12,NaN,2019-12-17,NaT,NaT,regular,NaT,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN


In [299]:
explo.check_null(data_df)

Count of null values:
cash_request_id                   0
amount                            0
CR_status                         0
CR_created_at                     0
CR_updated_at                     0
user_id                        2572
moderated_at                  10335
deleted_account_id            29521
reimbursement_date                0
cash_request_received_date     7945
money_back_date                8177
transfer_type                     0
send_at                        9724
recovery_status               24894
reco_creation                 24894
reco_last_update              24894
type                          11037
fee_status                    11037
total_amount                  11037
reason                        11037
fee_created_at                11037
fee_updated_at                11037
paid_at                       16656
from_date                     24328
to_date                       24328
charge_moment                 11037
dtype: int64


In [300]:
frames = cl.selecting_data_types(data_df)
num = frames[1]
cat = frames[0]
date = frames[2]

In [301]:
num.head()

,amount
0,100.0
1,100.0
2,100.0
3,99.0
4,100.0


In [302]:
explo.check_null(num)

Count of null values:
amount    0
dtype: int64


In [303]:
cat.head()

,CR_status,transfer_type,recovery_status,type,fee_status,reason,charge_moment,total_amount
0,rejected,regular,NaN,NaN,NaN,NaN,NaN,NaN
1,rejected,regular,NaN,NaN,NaN,NaN,NaN,NaN
2,rejected,regular,NaN,NaN,NaN,NaN,NaN,NaN
3,rejected,regular,NaN,NaN,NaN,NaN,NaN,NaN
4,rejected,regular,NaN,NaN,NaN,NaN,NaN,NaN


In [304]:
explo.check_null(cat)

Count of null values:
CR_status              0
transfer_type          0
recovery_status    24894
type               11037
fee_status         11037
reason             11037
charge_moment      11037
total_amount       11037
dtype: int64


In [305]:
date.head()

,CR_created_at,CR_updated_at,moderated_at,reimbursement_date,cash_request_received_date,money_back_date,send_at,reco_creation,reco_last_update,fee_created_at,fee_updated_at,paid_at,from_date,to_date
0,2019-12-10,2019-12-11,2019-12-11,2020-01-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,2019-12-10,2019-12-11,2019-12-11,2020-01-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,2019-12-10,2019-12-11,2019-12-11,2020-01-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,2019-12-10,2019-12-18,2019-12-18,2020-01-09,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,2020-05-06,2020-05-07,2020-05-07,2020-06-05,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [306]:
explo.check_null(date)

Count of null values:
CR_created_at                     0
CR_updated_at                     0
moderated_at                  10335
reimbursement_date                0
cash_request_received_date     7945
money_back_date                8177
send_at                        9724
reco_creation                 24894
reco_last_update              24894
fee_created_at                11037
fee_updated_at                11037
paid_at                       16656
from_date                     24328
to_date                       24328
dtype: int64


In [307]:
# Verify specific date columns' data types in cash_requests:
print("\nCheck data types of date columns:")
print(cash_requests[['created_at', 'updated_at', 'moderated_at', 'reimbursement_date', 
                     'cash_request_received_date', 'money_back_date', 'send_at', 'reco_creation', 'reco_last_update']].dtypes)

# Check for any NaT in date columns:
print("\nCheck NaT (missing values) in date columns:")
print(cash_requests[['created_at', 'updated_at', 'moderated_at', 'reimbursement_date', 
                     'cash_request_received_date', 'money_back_date', 'send_at', 'reco_creation', 'reco_last_update']].isna().sum())


Check data types of date columns:
created_at                    datetime64[ns]
updated_at                    datetime64[ns]
moderated_at                  datetime64[ns]
reimbursement_date            datetime64[ns]
cash_request_received_date    datetime64[ns]
money_back_date               datetime64[ns]
send_at                       datetime64[ns]
reco_creation                 datetime64[ns]
reco_last_update              datetime64[ns]
dtype: object

Check NaT (missing values) in date columns:
created_at                        0
updated_at                        0
moderated_at                   7935
reimbursement_date                0
cash_request_received_date     7681
money_back_date                7427
send_at                        7504
reco_creation                 20640
reco_last_update              20640
dtype: int64


In [308]:
df_time_between_CR_and_money_back = cl.process_date_columns(date, "CR_created_at", "money_back_date")
df_time_between_CR_and_money_back.head()

,CR_created_at,money_back_date,days_difference
5,2020-05-23,2020-07-06,44
11,2020-05-14,2020-06-03,20
13,2020-05-22,2020-06-13,22
43,2020-05-08,2020-06-10,33
48,2020-10-23,2020-11-06,14


In [309]:
explo.check(df_time_between_CR_and_money_back)

Number of columns: 3 amd rows: 23917

Data types:
CR_created_at      datetime64[ns]
money_back_date    datetime64[ns]
days_difference             int64
dtype: object

Unique values count:
CR_created_at      314
money_back_date    315
days_difference    206
dtype: int64

This columns apear to be categroical:
 Index([], dtype='object')

Unique value count for categorical columns:
Count of null values:
CR_created_at      0
money_back_date    0
days_difference    0
dtype: int64

Count of duplicated values:
18132


In [310]:
df_time_between_fee_created_and_paid_at_date = cl.process_date_columns(date, "fee_created_at", "paid_at")
df_time_between_fee_created_and_paid_at_date.head(20)

,fee_created_at,paid_at,days_difference
48,2020-10-23,2020-11-06,14
170,2020-10-17,2020-10-17,0
171,2020-10-15,2020-10-31,16
233,2020-06-28,2020-06-28,0
307,2020-07-05,2020-08-06,32
330,2020-07-07,2020-07-07,0
331,2020-07-08,2020-07-08,0
332,2020-10-04,2020-10-30,26
333,2020-09-04,2020-10-30,56
423,2020-10-09,2020-11-06,28


In [311]:
explo.check(df_time_between_fee_created_and_paid_at_date)

Number of columns: 3 amd rows: 15438

Data types:
fee_created_at     datetime64[ns]
paid_at            datetime64[ns]
days_difference             int64
dtype: object

Unique values count:
fee_created_at     132
paid_at            229
days_difference    165
dtype: int64

This columns apear to be categroical:
 Index([], dtype='object')

Unique value count for categorical columns:
Count of null values:
fee_created_at     0
paid_at            0
days_difference    0
dtype: int64

Count of duplicated values:
11315


In [312]:
# We can divide the columns between us and ecah work in a set of columns performing an 
# univariate analysis (frequency plots, box plots, doughnut, histograms   or any other you concider)
#For numercial plots also create report in mean , meadian, std, and arrive to conclusions 

In [313]:
print(cat.columns)


Index(['CR_status', 'transfer_type', 'recovery_status', 'type', 'fee_status',
       'reason', 'charge_moment', 'total_amount'],
      dtype='object')


In [314]:
invalid_dates = cat[~cat['charge_moment'].apply(pd.to_datetime, errors='coerce').notna()]
print(invalid_dates)


               CR_status transfer_type recovery_status             type  \
0               rejected       regular             NaN              NaN   
1               rejected       regular             NaN              NaN   
2               rejected       regular             NaN              NaN   
3               rejected       regular             NaN              NaN   
4               rejected       regular             NaN              NaN   
...                  ...           ...             ...              ...   
32089         money_back       instant             NaN  instant_payment   
32090         money_back       instant             NaN         postpone   
32091         money_back       instant             NaN  instant_payment   
32092  direct_debit_sent       instant             NaN         postpone   
32093  direct_debit_sent       instant             NaN  instant_payment   

      fee_status                        reason charge_moment  total_amount  
0            NaN      

In [315]:
missing_count = cat[cat['charge_moment'].isna()].shape[0]
print(f'Number of rows with missing charge_moment: {missing_count}')


Number of rows with missing charge_moment: 11037


In [316]:
# Convert 'charge_moment' to datetime (already done in your previous steps)
cat['charge_moment'] = pd.to_datetime(cat['charge_moment'], errors='coerce')

# Calculate the median date of valid entries
median_date = cat['charge_moment'].dropna().median()

# Fill missing or invalid dates with the median
cat['charge_moment'] = cat['charge_moment'].fillna(median_date)

# Optionally: check the median date you calculated
print(f"Median date for imputation: {median_date}")

Median date for imputation: NaT


In [317]:
import pandas as pd

# Ensure 'charge_moment' is in datetime format
cat['charge_moment'] = pd.to_datetime(cat['charge_moment'])

# Step 1: Define cohorts based on the month of the first cash request
cat['cohort_month'] = cat['charge_moment'].dt.to_period('M')

# Step 2: Clean the 'reason' column (strip whitespace and convert to lowercase)
cat['reason'] = cat['reason'].str.strip().str.lower()

# Step 3: Define payment incident reasons (in lowercase)
incident_reasons = ['rejected direct debit', 'month delay on payment']

# Step 4: Filter rows where 'reason' matches one of the incident reasons, and drop NaN values
incident_data = cat[cat['reason'].isin(incident_reasons)].dropna(subset=['reason'])

# Check if any incidents are found after filtering
if incident_data.empty:
    print("Incident data is empty. Inspecting data:")
    print(cat[['reason', 'cohort_month']].head(20))  # Check the first 20 rows for any reason matches

# Step 5: Group by cohort (cohort_month) and calculate the number of incidents for each cohort
incident_rate = incident_data.groupby('cohort_month').size().reset_index(name='incident_count')

# Step 6: Count the total number of requests for each cohort (including both completed and incomplete requests)
total_requests = cat.groupby('cohort_month').size().reset_index(name='total_requests')

# Step 7: Merge the incident count and total requests data
cohort_data = pd.merge(incident_rate, total_requests, on='cohort_month', how='left')

# Step 8: Calculate incident rate (number of incidents divided by the total number of requests in that cohort)
cohort_data['incident_rate'] = cohort_data['incident_count'] / cohort_data['total_requests']

# Step 9: Fill any missing values in incident_count with 0 (no incidents for that cohort)
cohort_data['incident_count'] = cohort_data['incident_count'].fillna(0)

# Display the results
print(cohort_data)


Empty DataFrame
Columns: [incident_count, cohort_month, total_requests, incident_rate]
Index: []
